<a href="https://colab.research.google.com/github/CerberDev/IA.2A-AgenteIALeitorCSV.ipynb/blob/main/AgenteIALeitorCSV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#(Célula de Texto para a CÉLULA 1: Instalação)
⚙️ Célula 1: Instalação das Dependências
Esta célula prepara o ambiente do Google Colab, instalando todas as bibliotecas Python necessárias para que o nosso agente funcione corretamente.

Cada biblioteca tem um papel fundamental:

openai, langchain, langchain-openai, langchain-experimental: Formam o núcleo da nossa aplicação, o framework LangChain, que nos permite construir o agente e conectá-lo ao modelo de linguagem.
pandas: A principal ferramenta para manipulação de dados. É usada para carregar os arquivos CSV, limpar os dados e realizar as análises.
tabulate: Uma dependência do LangChain, usada internamente para formatar tabelas de dados.
unidecode: Uma biblioteca crucial que adicionamos para "limpar" os nomes das colunas,

In [ ]:
# ===========================================================================================
# CÉLULA 1: INSTALAÇÃO DE DEPENDÊNCIAS
# ------------------------------------------------------------------------------------------
# Adicionamos 'unidecode' para nos ajudar a limpar os nomes das colunas (remover acentos).
# ==========================================================================================
!pip install openai langchain langchain-openai langchain-experimental pandas tabulate unidecode -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.8/235.8 kB 4.7 MB/s eta 0:00:00


#📊 Célula 2: Carga, Limpeza e União dos Dados
Esta é a célula de preparação de dados, uma das mais importantes do projeto. Ela executa uma sequência de tarefas para deixar os dados prontos para a análise da IA:

Coleta Segura da Chave: Pede a chave da API usando getpass, para que ela não fique visível na tela.
Upload e Descompactação: Permite o upload do arquivo .zip e extrai os arquivos CSV para uma pasta temporária.
Leitura dos CSVs: Carrega os dois arquivos (Cabecalho e Itens) em DataFrames do Pandas, especificando a codificação encoding='latin-1' para evitar erros de leitura.
Limpeza dos Nomes das Colunas: Aplica a função clean_col_names para padronizar todos os nomes de colunas. Ela converte para minúsculas, remove acentos e substitui espaços por _. Este passo é vital para que o agente de IA consiga referenciar as colunas de forma consistente e sem erros.
União dos Dados (Merge): Combina os dois DataFrames em uma única tabela, df_completo, usando a coluna chave_de_acesso como elo de ligação. Isso simplifica imensamente a tarefa do agente, que agora só precisa consultar uma única fonte de dados.

In [ ]:
# ==============================================================================
# CÉLULA 2: CARREGAMENTO, LIMPEZA E UNIÃO DOS DADOS
# ------------------------------------------------------------------------------
# MUDANÇAS FINAIS:
# 1. Usamos o nome correto da coluna de união: 'CHAVE DE ACESSO'.
# 2. Criamos uma função 'clean_col_names' para padronizar TODOS os nomes de colunas:
#    - Converte para minúsculas.
#    - Remove acentos.
#    - Substitui espaços e caracteres especiais por underscore (_).
#    Isso aumenta drasticamente a chance de sucesso do agente.
# ==============================================================================
import pandas as pd
import os
import zipfile
import re
from getpass import getpass
from google.colab import files
from IPython.display import display
from unidecode import unidecode

# Função para limpar os nomes das colunas
def clean_col_names(df):
    new_columns = []
    for col in df.columns:
        # Remove acentos
        cleaned_col = unidecode(col)
        # Converte para minúsculas
        cleaned_col = cleaned_col.lower()
        # Substitui caracteres não alfanuméricos por underscore
        cleaned_col = re.sub(r'[^0-9a-zA-Z]+', '_', cleaned_col)
        # Remove underscores no início ou fim
        cleaned_col = cleaned_col.strip('_')
        new_columns.append(cleaned_col)
    df.columns = new_columns
    return df

# 1. Obter a chave da API de forma segura
api_key = getpass("🔑 Cole sua chave da API do OpenRouter aqui: ")
os.environ["OPENAI_API_KEY"] = api_key

# 2. Fazer o upload do arquivo .zip
print("\nPor favor, faça o upload do arquivo .zip contendo os CSVs das notas fiscais.")
uploaded = files.upload()

if not uploaded:
    print("\n❌ Nenhuma arquivo foi enviado. A execução foi interrompida.")
else:
    zip_filename = list(uploaded.keys())[0]
    # 3. Descompactar o arquivo
    with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
        zip_ref.extractall('dados_nfs')
    print(f"\n✅ Arquivo '{zip_filename}' descompactado com sucesso na pasta 'dados_nfs'.")

    # 4. Carregar os CSVs
    try:
        path_cabecalho = 'dados_nfs/202401_NFs_Cabecalho.csv'
        path_itens = 'dados_nfs/202401_NFs_Itens.csv'

        df_cabecalho = pd.read_csv(path_cabecalho, encoding='latin-1')
        df_itens = pd.read_csv(path_itens, encoding='latin-1')

        # 5. Limpar os nomes das colunas de AMBOS os dataframes
        df_cabecalho = clean_col_names(df_cabecalho)
        df_itens = clean_col_names(df_itens)
        print("\n✅ Nomes de colunas limpos e padronizados.")
        print("\nNovas colunas do cabeçalho:", df_cabecalho.columns.tolist())
        print("Novas colunas dos itens:", df_itens.columns.tolist())

        # 6. Unir os dois dataframes usando a chave correta e limpa: 'chave_de_acesso'
        if 'chave_de_acesso' in df_cabecalho.columns and 'chave_de_acesso' in df_itens.columns:
            # Remove colunas duplicadas do df_cabecalho antes do merge para evitar conflitos
            cols_to_drop = [col for col in df_itens.columns if col in df_cabecalho.columns and col != 'chave_de_acesso']
            df_cabecalho_to_merge = df_cabecalho.drop(columns=cols_to_drop)

            df_completo = pd.merge(df_itens, df_cabecalho_to_merge, on='chave_de_acesso')
            print("\n✅ DataFrames de cabeçalho e itens unidos com sucesso!")
            print("\n--- Tabela Completa (df_completo) ---")
            display(df_completo.head())
        else:
            print("❌ Erro: A coluna 'chave_de_acesso' não foi encontrada após a limpeza.")
            df_completo = None

    except Exception as e:
        print(f"\n❌ Ocorreu um erro inesperado: {e}")
        df_completo = None

🔑 Cole sua chave da API do OpenRouter aqui: ··········

Por favor, faça o upload do arquivo .zip contendo os CSVs das notas fiscais.


Saving 202401_NFs.zip to 202401_NFs (6).zip

✅ Arquivo '202401_NFs (6).zip' descompactado com sucesso na pasta 'dados_nfs'.

✅ Nomes de colunas limpos e padronizados.

Novas colunas do cabeçalho: ['chave_de_acesso', 'modelo', 'sarie', 'namero', 'natureza_da_operaaao', 'data_emissao', 'evento_mais_recente', 'data_hora_evento_mais_recente', 'cpf_cnpj_emitente', 'razao_social_emitente', 'inscriaao_estadual_emitente', 'uf_emitente', 'municapio_emitente', 'cnpj_destinatario', 'nome_destinatario', 'uf_destinatario', 'indicador_ie_destinatario', 'destino_da_operaaao', 'consumidor_final', 'presenaa_do_comprador', 'valor_nota_fiscal']
Novas colunas dos itens: ['chave_de_acesso', 'modelo', 'sarie', 'namero', 'natureza_da_operaaao', 'data_emissao', 'cpf_cnpj_emitente', 'razao_social_emitente', 'inscriaao_estadual_emitente', 'uf_emitente', 'municapio_emitente', 'cnpj_destinatario', 'nome_destinatario', 'uf_destinatario', 'indicador_ie_destinatario', 'destino_da_operaaao', 'consumidor_final', 'pres

,chave_de_acesso,modelo,sarie,namero,natureza_da_operaaao,data_emissao,cpf_cnpj_emitente,razao_social_emitente,inscriaao_estadual_emitente,uf_emitente,municapio_emitente,cnpj_destinatario,nome_destinatario,uf_destinatario,indicador_ie_destinatario,destino_da_operaaao,consumidor_final,presenaa_do_comprador,namero_produto,descriaao_do_produto_serviao,cadigo_ncm_sh,ncm_sh_tipo_de_produto,cfop,quantidade,unidade,valor_unitario,valor_total,evento_mais_recente,data_hora_evento_mais_recente,valor_nota_fiscal
0,41240106267630001509550010035101291224888487,55 - NF-E EMITIDA EM SUBSTITUIÃÃO AO MODELO ...,1,3510129,Outras Entradas - Dev Remessa Escola,2024-01-18 07:10:39,06267630001509,COMPANHIA BRASILEIRA DE EDUC. E SIST. DE ENS. ...,9085104702,PR,CURITIBA,394429021965,COMANDO DA AERONAUTICA,PA,CONTRIBUINTE ISENTO,2 - OPERAÃÃO INTERESTADUAL,1 - CONSUMIDOR FINAL,"9 - OPERAÃÃO NÃO PRESENCIAL, OUTROS",1,COLECAO SPE EF1 4ANO VOL 1 AL,49019900,"Outros livros, brochuras e impressos semelhantes",2949,1.0,UNIDAD,522.5,522.5,AutorizaÃ§Ã£o de Uso,2024-01-18 07:10:58,522.5
1,50240129843878000170550010000025251000181553,55 - NF-E EMITIDA EM SUBSTITUIÃÃO AO MODELO ...,1,2525,VENDA DE MERCADORIA FORA DO ESTADO,2024-01-26 11:24:42,29843878000170,V CALDI PEREIRA PECAS E SERVICOS DIESEL,284290777,MS,COXIM,9615848000108,4 BATALHAO LOGISTICO,RS,NÃO CONTRIBUINTE,2 - OPERAÃÃO INTERESTADUAL,1 - CONSUMIDOR FINAL,1 - OPERAÃÃO PRESENCIAL,1,LANTERNA TATERAL CARRETA LED,85122021,Luzes fixas para automÃ³veis e outros ciclos,6403,4.0,UNIDAD,39.9,159.6,AutorizaÃ§Ã£o de Uso,2024-01-26 11:24:43,499.0
2,50240129843878000170550010000025251000181553,55 - NF-E EMITIDA EM SUBSTITUIÃÃO AO MODELO ...,1,2525,VENDA DE MERCADORIA FORA DO ESTADO,2024-01-26 11:24:42,29843878000170,V CALDI PEREIRA PECAS E SERVICOS DIESEL,284290777,MS,COXIM,9615848000108,4 BATALHAO LOGISTICO,RS,NÃO CONTRIBUINTE,2 - OPERAÃÃO INTERESTADUAL,1 - CONSUMIDOR FINAL,1 - OPERAÃÃO PRESENCIAL,2,CINEMATICO RODO-AR,73071920,AcessÃ³rios moldados para tubos de aÃ§o,6403,4.0,PEÃA,75.0,300.0,AutorizaÃ§Ã£o de Uso,2024-01-26 11:24:43,499.0
3,50240129843878000170550010000025251000181553,55 - NF-E EMITIDA EM SUBSTITUIÃÃO AO MODELO ...,1,2525,VENDA DE MERCADORIA FORA DO ESTADO,2024-01-26 11:24:42,29843878000170,V CALDI PEREIRA PECAS E SERVICOS DIESEL,284290777,MS,COXIM,9615848000108,4 BATALHAO LOGISTICO,RS,NÃO CONTRIBUINTE,2 - OPERAÃÃO INTERESTADUAL,1 - CONSUMIDOR FINAL,1 - OPERAÃÃO PRESENCIAL,3,ESPIRAL NYLON CABINE AMARELO,39173100,Tubos flexÃ­veis podendo suportar uma pressÃ£o...,6403,1.0,PEÃA,39.9,39.9,AutorizaÃ§Ã£o de Uso,2024-01-26 11:24:43,499.0
4,50240112977901000117550010000051831659469117,55 - NF-E EMITIDA EM SUBSTITUIÃÃO AO MODELO ...,1,5183,Venda de mercadorias,2024-01-22 11:01:09,12977901000117,MOSKO LTDA EPP,284058734,MS,CAMPO GRANDE,15461510000133,FUNDACAO UNIVERSIDADE FEDERAL DE MS,MS,NÃO CONTRIBUINTE,1 - OPERAÃÃO INTERNA,1 - CONSUMIDOR FINAL,0 - NÃO SE APLICA,1,"AGUA MINERAL NATURAL, TIPO SEM GAS MATERIAL EM...",22011000,"Ãguas minerais e Ã¡guas gaseificadas, nÃ£o ad...",5405,9.0,GL,37.5,337.5,AutorizaÃ§Ã£o de Uso,2024-01-22 10:01:09,337.5


#🤖 Célula 3: Construção e Treinamento do Agente de IA
Nesta célula, construímos o "cérebro" da nossa aplicação. É aqui que o agente de IA é configurado e inicializado com suas regras e capacidades.

Configuração do Modelo (LLM): Definimos qual modelo de linguagem será usado. Escolhemos o google/gemini-flash-1.5 por sua alta capacidade de seguir instruções e por seu ótimo desempenho com o português.
Definição das Instruções (AGENT_PREFIX): Criamos um bloco de texto com instruções permanentes para o agente. Este é o "treinamento" principal que damos a ele, forçando-o a sempre responder em português e a sempre fornecer o nome do produto, não apenas seu código.
Criação do Agente: Usamos a função create_pandas_dataframe_agent para finalmente montar o agente, passando os três componentes principais:
O Modelo (llm).
Os Dados (df_completo).
As Instruções (prefix=AGENT_PREFIX).
Habilitação da Autocorreção: Adicionamos o parâmetro agent_executor_kwargs={'handle_parsing_errors': True}. Esta é uma configuração avançada que torna o agente mais robusto, permitindo que ele se corrija caso cometa pequenos erros de formatação na resposta final, em vez de quebrar a execução.

In [ ]:
# ==============================================================================
# CÉLULA 3:
# ------------------------------------------------------------------------------
# SOLUÇÃO DEFINITIVA PARA O 'Output parsing error':
# A mensagem de erro nos instrui a usar 'handle_parsing_errors=True'. A forma
# correta de fazer isso nas versões recentes do LangChain é passando este
# parâmetro dentro do dicionário 'agent_executor_kwargs'. Isso torna o agente
# tolerante a pequenos erros de formatação na resposta final, permitindo que
# ele se corrija em vez de quebrar.
# ==============================================================================
from langchain_openai import ChatOpenAI
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent

if 'df_completo' in locals() and df_completo is not None:
    # 1. Configurar o LLM
    llm = ChatOpenAI(
        model_name="google/gemini-flash-1.5",
        openai_api_base="https://openrouter.ai/api/v1",
        temperature=0,
    )

    # 2. Definir as instruções permanentes para o início do prompt
    AGENT_PREFIX = """
Você é um assistente especialista em análise de dados com pandas que se comunica exclusivamente em português do Brasil.
Você recebeu um dataframe com dados de notas fiscais.
Siga estas regras rigorosamente:

INSTRUÇÕES FINAIS:
1. Responda SEMPRE em português do Brasil, em uma frase completa e natural.
2. Quando a pergunta for sobre um 'produto', sua resposta final DEVE conter o nome/descrição do produto (use a coluna 'descriaao_do_produto_serviao'), não apenas seu ID ou número.
3. Analise a pergunta do usuário com atenção antes de executar qualquer código.
"""

    # 3. Criar o agente com o manipulador de erros de parsing ativado
    agent = create_pandas_dataframe_agent(
        llm,
        df_completo,
        prefix=AGENT_PREFIX,
        verbose=False,
        allow_dangerous_code=True,
        # Habilitando o modo de autocorreção da maneira correta:
        agent_executor_kwargs={'handle_parsing_errors': True},
    )
    print("✅ Agente final, com autocorreção, criado com sucesso!")

else:
    print("❌ O DataFrame completo não foi criado. Não é possível criar o agente.")

✅ Agente final, com autocorreção, criado com sucesso!


#💬 Célula 4: Interface de Interação com o Usuário
Esta é a célula final, responsável pela interação com o usuário. Ela cria uma interface de chat simples e contém a lógica para "conversar" com o agente que criamos na célula anterior.

Criação dos Widgets: Usa a biblioteca ipywidgets para desenhar a caixa de texto onde você digita a pergunta e o botão "Perguntar ao Agente".
Lógica do Botão (on_button_clicked): Define o que acontece quando o botão é clicado:
A pergunta digitada é enviada para o agente através da função agent.invoke().
O sistema aguarda a resposta do agente.
"Rede de Segurança": Antes de exibir a resposta, o código faz uma verificação final por palavras-chave que indiquem falha ou confusão (como "não sei", "não consigo"). Se o agente falhar, em vez de mostrar uma resposta ruim, o sistema exibe uma mensagem útil, ensinando o usuário a fazer uma pergunta melhor. Isso torna a experiência de uso muito mais amigável e à prova de falhas.

In [ ]:
# ==============================================================================
# CÉLULA INTERFACE DE CHAT:
# ------------------------------------------------------------------------------
# MUDANÇA: Adicionamos uma lógica de verificação na resposta do agente.
# Se a resposta indicar que o agente ficou confuso ou não conseguiu encontrar
# a informação, o programa não mostra a resposta ruim. Em vez disso, ele exibe
# uma mensagem útil, pedindo para o usuário refazer a pergunta de forma mais
# específica e dando exemplos de como fazer isso.
# ==============================================================================
import ipywidgets as widgets
from IPython.display import display, Markdown, clear_output

# Widgets da interface
question_input = widgets.Text(
    value='',
    placeholder='Digite sua pergunta aqui...',
    description='Pergunta:',
    layout={'width': '80%'}
)
run_button = widgets.Button(
    description="💬 Perguntar ao Agente",
    button_style='info',
)
output_area = widgets.Output()

# --- INÍCIO DA LÓGICA MELHORADA ---
# Função para lidar com o clique do botão
def on_button_clicked(b):
    with output_area:
        clear_output()
        question = question_input.value
        if not question:
            print("Por favor, digite uma pergunta.")
            return

        print(f"🤔 Sua pergunta: {question}")
        print("\n🤖 O agente está pensando...")
        print("---")

        try:
            # Chamamos o agente normalmente
            response = agent.invoke({"input": question})
            output_text = response['output']

            # Lista de palavras-chave que indicam que o agente falhou ou ficou confuso
            failure_keywords = [
                "não sei",
                "nao sei",
                "não consigo",
                "nao consigo",
                "não posso responder",
                "i cannot",
                "i don't know",
                "keyerror",
                "não foi possível encontrar"
            ]

            # Verificamos se alguma palavra-chave de falha está na resposta (em minúsculas)
            is_failure = any(keyword in output_text.lower() for keyword in failure_keywords)

            if is_failure:
                # Se o agente falhou, guiamos o usuário
                print("O agente não teve certeza de como responder. 🤔")
                print("Tente fazer uma pergunta mais específica. Aqui estão algumas dicas:\n")
                display(Markdown("""
                * **Especifique o que você quer:** Em vez de "qual o melhor produto?", pergunte "qual o **nome** do produto com a **maior soma de quantidade**?".
                * **Peça por listas ou tabelas:** "Liste os 5 produtos mais caros em uma tabela, mostrando o nome e o valor total".
                * **Use os nomes das colunas (se souber):** "Qual a média da coluna `valor_total` para o `razao_social_emitente` 'NOME DA EMPRESA'?"
                """))
            else:
                # Se a resposta parece boa, exiba-a
                display(Markdown(output_text))

        except Exception as e:
            # Captura de erros de execução
            print(f"Ocorreu um erro durante a execução do agente: {e}")
            print("\nIsso pode acontecer se a pergunta for muito complexa ou ambígua. Tente simplificá-la.")
# --- FIM DA LÓGICA MELHORADA ---


# Vincula a função ao botão e exibe a interface
if 'agent' in locals():
    run_button.on_click(on_button_clicked)
    display(widgets.VBox([question_input, run_button, output_area]))
else:
    print("❌ O agente não foi inicializado. Rode as células anteriores.")